## Create a SV dataframe where the headers are 'deletions', 'duplications', 'insertions' and 'inversions' per a decided genomic window. The max SV_length and min SV_length can also be decided here.

In [3]:
import math
import re

def make_headers(window):
    headers = open("SV_data/SV_headers", "w")
    chrom_dict = {}
    window_dict= {}
    for i in range(25):
        chrom_dict["LG"+str(i+1)]=0
    with open(r"/workspace/hramzr/2_Phd_PROJECT/plotting/chromosomefile.genome", "r") as f:
        for line in f:
            chrom_dict[str(line.split()[0])]=int(line.split()[1])
    headers.write("indv")
    for i in range(25):
        for j in range (math.floor(int(chrom_dict["LG"+str(i+1)])/window)):
            window_dict["LG"+str(i+1)+"_"+str(((j+1)*window))]="0"
            for value in ['dels', 'dups', 'ins', 'invs']:
                headers.write(" LG"+str(i+1)+"_"+value+"_"+str(((j+1)*window)))
    headers.write("\n")
    headers.close()
    return window_dict
    
    
def make_body(window, window_dict, max_svlen, min_svlen):
    indvs = open(r"/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/help_files/indvs", "r")
    body = open("SV_data/SV_body", "w")
    for indv in indvs:
        body.write(indv.strip()+" ")
        bin_size=1
        chrom="LG1"
        indv_window_dict=window_dict
        with open(r"/workspace/hramzr/2_Phd_PROJECT/VarCallingWGS/parliament2_calling/"+ \
                   str(indv.split()[0])+"/outputs/"+str(indv.split()[0])+"_q20.combined.genotyped.vcf", "r") as f:
            chrom_dict={}
            for i in range(25):
                chrom_dict["LG"+str(i+1)]={'DEL':0, 'DUP':0, 'INS':0, 'INV':0}
            for line in f:
                if "#" not in line:
                    start=line.split()[1]
                    if chrom != line.split()[0][3:]:
                        bin_size=1
                    if ((int(start)/window)>bin_size):
                        window_size=(bin_size*window)
                        indv_window_dict[chrom+"_"+str(window_size)]=chrom_dict[chrom]
                        for i in range(25):
                            chrom_dict["LG"+str(i+1)]={'DEL':0, 'DUP':0, 'INS':0, 'INV':0}
                        bin_size+=1
                    end, chrom, start, svlen, svtype=line.split()[7].split(";")[6][4:], line.split()[0][3:], line.split()[1], \
                    line.split(";")[2][7:], line.split(";")[3][7:]
                    if round(float(svlen))<max_svlen and round(float(svlen))>min_svlen:
                        chrom_dict[chrom][svtype]+=1
        write_window_dict_values(indv_window_dict, body)                
        body.write("\n")
    body.close()
    
def write_window_dict_values(indv_window_dict, body):
    for row in indv_window_dict.values():
        if row == "0":
            body.write("0 0 0 0 ")
        else:
            row=re.sub("[^0-9]", " ", str(row))
            row=' '.join(row.split())
            body.write(row+" ")
    
def merge():
    merge = open("SV_data/SV_dataframe_50kb_1kb_5", "w")
    with open(r"SV_data/SV_body", "r") as f:
        with open(r"SV_data/SV_headers", "r") as z:
            for line in z:
                merge.write(line)
            for line in f:
                merge.write(line)
                
if __name__ == "__main__":
    window = 50000
    max_svlen=1000
    min_svlen=5
    window_dict=make_headers(window)
    make_body(window, window_dict, max_svlen, min_svlen)
    merge()